# Creates a Yelp POSTGRESSQL database
 - Uses the open source Yelp dataset and inserts json files into SQL tables

## Table of contents 
- importing json into python using pandas tools
- importing json to postgresql database

## TODO
- write sample queries extracting from json to ensure it works
- import every Yelp JSON
- refactor code 

In [1]:
import json 
import pandas as pd 
from re import sub

from sqlalchemy import create_engine,text
from sqlalchemy.engine.url import URL 
import psycopg2 #postgressql driver

In [33]:
not False and True

True

In [30]:
def get_json_data(json_file_path,
    x_lines_to_read = -1,
    is_df_read_all_lines=True):
    with open(json_file_path) as json_file:
        if x_lines_to_read is -1:
            json_list = json_file.readlines()
        elif x_lines_to_read>1:
            json_list = [json_file.read_line() for i in range(x_lines_to_read)]
        else:
            json_return_obj = json_file.read_line()

        if is_df_read_all_lines and x_lines_to_read>1:
            json_data = [json.loads(json_line) for json_line in json_list] 
            json_df = pd.DataFrame(json_data)
            return json_df
        elif not  x_lines_to_read>1:
        else:
            json_return_obj = 


#format the columns/fields that will be jsonb
def stringify_json_records(series):
    '''
    converts a series of dicts into a stringified 
    set of json records (which is necessary format for json storage)
    for example single quotes are replaced by double quotes
    and the dict is converted into a single quote string.
    '''
    if series.apply(lambda x: x is None or type(x) is dict).all():
       json_string_records = (
           series
           .apply(json.dumps)
           .apply(_restringify_nested_json_records) 
       )
    else:
        print("the series {} are not all dict records".format(series.name))
        json_string_records = series
    return json_string_records

def _restringify_nested_json_records(stringified_record):
    '''
    yelp open dataset has a nested (depth 2) json that is a string (invalid) and has single quotes
    rather than double quotes and should not have . need to change to be considered valid 
    json and inserted into postgressql table.
    '''
    #TODO: clean up with re.sub or replace booleans and nulls with proper json syntax (e.g., False is false and None is null)
    corrected_record = (
        stringified_record
        .replace('\"{','{') #nested json object should not have any quotes
        .replace('}\"','}')
        .replace('\'','\"') #change single to double quotes
        .replace('False,','\"False\",') #stringify booleans if not already
        .replace('True,','\"True\",')
        .replace('False}','\"False\"}') #end of dict boolean
        .replace('True}','\"True\"}')
        .replace('\"u\"','\"') #some records had double double quotes (ie "u"no"")
        .replace('\"\"','\"')
        .replace('None,','"None",') #stringify nones
        .replace('None}','"None"}')
    )
    return corrected_record


#functions to format for SQL-ready strings


def convert_to_sql_str(obj):
    '''
    if string : escape single quotes and/or make SQL quotes ($$), if not string : convert
    to string
    (see https://www.postgresqltutorial.com/dollar-quoted-string-constants/#:~:text=In%20PostgreSQL%2C%20you%20use%20single,doubling%20up%20the%20single%20quote.)
    '''
    if type(obj) is str:
        obj_str = "$$" + obj.replace("$","$ ") + "$$" #if $ at end of string, will have have invalid triple $$$
    else:
        obj_str = str(obj)
    return obj_str

def make_sql_str_values(sql_str_list):
    '''
    make valid values (ie record) for INSERT statement

    sql_str_list : iterable of strings
    '''
    #convert every element to valid sql statements
    #join fields with ',' and wrap in '()' to 
    sql_str_values = "({})".format(",".join(sql_str_list))
    return sql_str_values 

SyntaxError: invalid syntax (<ipython-input-30-246c6a5800d4>, line 2)

In [3]:
yelp_data_path = '/Users/michaelkranz/Documents/restaurant-app/data/yelp_dataset/'

yelp_json_filenames = {"tips":'yelp_academic_dataset_tip.json',"reviews":'yelp_academic_dataset_review.json',
"business_info":'yelp_academic_dataset_business.json',"user":'yelp_academic_dataset_user.json'}

In [4]:
json_file = open(yelp_data_path+yelp_json_filenames['reviews'])

In [6]:
test_json = json_file.readline()

In [8]:
test_file = open(yelp_data_path+yelp_json_filenames['reviews'])

In [16]:
eval(test_json).keys()

dict_keys(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'])

In [9]:
test_json = test_file.readline()

In [11]:
text(test_json)

{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It's what real estate agents would call \"cozy\" or \"charming\" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it's not kid friendly at all. Seriously, don't bring them.\n* the security is n

In [ ]:
#%%timeit
with open(yelp_data_path+yelp_json_filenames['business_info']) as json_file:
    json_list = json_file.readlines()
json_data = [json.loads(json_line) for json_line in json_list] 
json_df = pd.DataFrame(json_data)

In [ ]:
#upload json to postgresSQL table
json_df.head(1)

In [49]:
json_df['attributes'] = stringify_json_records(json_df['attributes'])
json_df['hours'] = stringify_json_records(json_df['hours'])

In [ ]:
json_str_list = (
    json_df
    .fillna('null') #null is syntax for SQL NAs
    .applymap(convert_to_sql_str)
    .apply(make_sql_str_values,axis=1)
    .values
)

### Storing JSON in PostgreSQL
- upon further research, it looks like we can store JSON directly in PostgreSQL without the traditional field format
    - [Storing JSON in PostgreSQL: A must-know feature] (https://www.blendo.co/blog/storing-json-in-postgresql/)
    - [Replacing EAV with JSONB in PostgreSQL*](https://coussej.github.io/2016/01/14/Replacing-EAV-with-JSONB-in-PostgreSQL/)

    *EAV = Entity,Attribute,Value (ie three tables connected with joins to get fields for the entity)


- find connection information 

>michaelkranz$ `psql`

>michaelkranz=# `CREATE DATABASE restaurants`

>michaelkranz=# `\c restaurants`

>restaurants=# `\conninfo`

> You are now connected to database "restaurants" as user "michaelkranz".
restaurants=# \conninfo
You are connected to database "restaurants" as user "michaelkranz" via socket in "/tmp" at port "5432".

- [CHAR and VARCHAR : no performance differences and character limits used to check and will return error if longer](https://www.postgresqltutorial.com/postgresql-char-varchar-text/)

- https://www.postgresql.org/docs/12/datatype-json.html

In [64]:
postgres_db_params = {'drivername': 'postgres',
                'database':'restaurants',
               'username': 'michaelkranz',
               'password': 'helloworld',
               'host': 'localhost',
               'port': 5432}

postgres_db_url = URL(**postgres_db_params)

In [65]:
engine = create_engine(postgres_db_url)

In [66]:
# store first level as a column but hours and attributes as jsonb format
business_info_create_table_str = '''
CREATE TABLE business_info (
    business_id VARCHAR PRIMARY KEY 
    ,name VARCHAR
    ,address VARCHAR
    ,city VARCHAR
    ,state VARCHAR
    ,postal_code VARCHAR  
    ,latitude FLOAT
    ,longitude FLOAT
    ,stars FLOAT(1)
    ,review_count INTEGER  
    ,is_open SMALLINT
    ,attributes JSONB
    ,categories VARCHAR
    ,hours JSONB
);
'''

In [18]:
# store first level as a column but hours and attributes as jsonb format
reviews_create_table_str = '''
CREATE TABLE reviews (
    review_gid PRIMARY KEY
    ,review_json JSONB
);
'''

In [26]:
test = {'review_gid':'PRIMARY KEY',
    'review':'test'}

In [29]:
'/n,'.join([key + ' ' + val for key,val in test.items()])

'review_gid PRIMARY KEY/n,review test'

In [ ]:
def _make_sql_field_str(field_dict,sep=' '):
    '''turns a field name (key) and datatypes (value)
    into SQL statement. Note- the default space 
    is for CREATE TABLE fxn.
    '''
    formatted_list = key + sep + val for key,val in field_dict.items()
    formatted_str = '/n,'.join(field_list)
    return formatted_str

def create_sql_table(table_name,field_dict):
    '''
    creates a table given a table name and set of variables with datatypes.
    outputs the list of table, column, and datatype dataframe.

    table_name : str
    field_dict : dict of field names (key) and datatypes(value)
    '''
    #create sql statement strings
    drop_table_str = f"DROP TABLE IF EXISTS {table_name};"

    field_str = _make_sql_field_str(field_dict)
    create_table_str = f'''
    CREATE TABLE {table_name} (
            {field_str}
        );
        '''

    table_desc_str = f'''
        SELECT 
            table_name
            ,column_name
            ,data_type
        FROM 
            information_schema.columns
        WHERE
            table_name='{table_name}'
            '''
    
    #execute sql statements
    with engine.connect() as conn:
        conn.execute(drop_table_str)
        conn.execute(create_table_str)

        table_desc = pd.read_sql(con=conn,sql=table_desc_str)

    return table_desc

def insert_sql_record(sql_str_value,table_name):
    '''
    insert a SQL string-formatted record (ie one SQL string) 
    or a list of SQL records (list of SQL strings)
    
     into a SQL table
    '''
    insert_str = '''
    INSERT INTO {}
    VALUES {};
    '''
    with engine.connect() as conn:
        if type(sql_str_value) is str:
            #text escapes special chars for you
            sql_record = text(
                insert_str.format(table_name,sql_str_value)
            )
            conn.execute(sql_record)
            i = 1
        elif type(sql_str_value) is list:
            for i,i_sql in enumerate(sql_str_value):
                #text escapes special chars for you
                sql_record = text(
                    insert_str.format(table_name,i_sql)
                    )
                conn.execute(sql_record)

    return f"{str(i)} SQL records inserted into {table_name}"

In [111]:
with engine.connect() as conn:
    for i,json_record in enumerate(json_str_list):
        #print(i)
        sql_record = text( #escapes special chars for you
            '''
            INSERT INTO business_info
            VALUES {};
            '''
            .format(json_record)
        )
        conn.execute(sql_record)

### Sample queries
- [Querying Json fields](https://www.postgresqltutorial.com/postgresql-json/)

In [127]:
#return BikeParking attribute
(
pd.read_sql(con=engine.connect(),sql='''
SELECT attributes -> 'BikeParking' as BikeParking
FROM business_info
'''
)
.head()
.values
)

array([['True'],
       [None],
       [None],
       ['True'],
       [None]], dtype=object)

In [132]:
#return BikeParking attribute
(
pd.read_sql(con=engine.connect(),sql='''
SELECT attributes ->> 'BikeParking' as BikeParking
FROM business_info
'''
)
.head()
)

,bikeparking
0,True
1,None
2,None
3,True
4,None


In [134]:
pd.read_sql(con=engine.connect(),sql='''
SELECT
    name
    ,address
    ,attributes ->> 'BikeParking' AS BikeParking
FROM business_info
WHERE attributes ->> 'BikeParking' = 'True' 
'''
)

,name,address,bikeparking
0,10913 Bailey Rd,The Range At Lake Norman,True
1,1015 Sharp Cir,Nevada House of Hose,True
2,"6870 S Rainbow Blvd, Ste 117",Green World Cleaners,True
3,2831 Parmenter St,Chocolate Shoppe Ice Cream,True
4,3131 Las Vegas Blvd,Manolo Blahnik,True
...,...,...,...
69804,829 E Washington Ave,Julep,True
69805,81 Front Street E,Starbucks,True
69806,"2487 S Gilbert Rd, Ste 102",Yao Fine Chinese Cuisine,True
69807,199 College St,Steak & Cheese & Quick Pita Restaurant,True
